In [ ]:
!pip install --upgrade pip numpy pandas librosa tensorflow keras scikit-learn matplotlib seaborn soundfile audioread

In [ ]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
import soundfile as sf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# Set visualization style
sns.set(style='white', context='notebook', palette='deep')

# Paths
DATASET_PATHS = {
    "train": r"E:\Minor_project\FakeAudioDetector\DataSet\training",
    "val": r"E:\Minor_project\FakeAudioDetector\DataSet\validation",
    "test": r"E:\Minor_project\FakeAudioDetector\DataSet\testing"
}
MODEL_PATH = r"E:\Minor_project\FakeAudioDetector\fake_audio_model.h5"
NEW_AUDIO_PATH = r"E:\Minor_project\FakeAudioDetector\AudioFile\file16.mp3_16k.mp3_norm.mp3_mono.mp3_silence.mp3_2sec.mp3"

# Extract Features (Optimized: Memory-efficient processing)
def extract_features(file_path, max_pad_len=174):
    try:
        audio, sr = librosa.load(file_path, sr=22050, mono=True, duration=2.0)  # Loads only 2 seconds to reduce RAM usage
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfccs = np.pad(mfccs, ((0, 0), (0, max(0, max_pad_len - mfccs.shape[1]))), mode='constant')[:, :max_pad_len]
        return mfccs
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Load Dataset (Optimized: Avoids redundant loading)
def load_dataset(folder_path):
    X, Y = [], []
    label_map = {label: idx for idx, label in enumerate(os.listdir(folder_path))}
    
    for label, idx in label_map.items():
        subfolder = os.path.join(folder_path, label)
        for file in os.listdir(subfolder):
            file_path = os.path.join(subfolder, file)
            if file.endswith(('.wav', '.mp3')):
                features = extract_features(file_path)
                if features is not None:
                    X.append(features)
                    Y.append(idx)

    return np.array(X), to_categorical(np.array(Y), len(label_map))

# Load Data Efficiently
X_train, Y_train = load_dataset(DATASET_PATHS["train"])
X_val, Y_val = load_dataset(DATASET_PATHS["val"])
X_test, Y_test = load_dataset(DATASET_PATHS["test"])

# Reshape for CNN
X_train = X_train.reshape(-1, 40, 174, 1)
X_val = X_val.reshape(-1, 40, 174, 1)
X_test = X_test.reshape(-1, 40, 174, 1)

# Build Optimized CNN Model
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(40, 174, 1)),
        MaxPool2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPool2D((2, 2)),
        Dropout(0.3),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(2, activation='softmax')
    ])
    
    model.compile(optimizer=Adam(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Load or Train Model
if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    print("Loaded saved model.")
else:
    model = build_model()
    model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=15, batch_size=16, verbose=2)
    model.save(MODEL_PATH)
    print("Model trained and saved.")

# Evaluate Model Performance
def evaluate_model():
    Y_pred = model.predict(X_test)
    cm = confusion_matrix(Y_test.argmax(axis=1), Y_pred.argmax(axis=1))

    print("Classification Report:\n", classification_report(Y_test.argmax(axis=1), Y_pred.argmax(axis=1)))
    
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=["Fake", "Real"], yticklabels=["Fake", "Real"])
    plt.title('Confusion Matrix'); plt.xlabel('Predicted'); plt.ylabel('True')
    plt.show()

# Predict Fake or Real for a New Audio File
def predict_audio(audio_path):
    features = extract_features(audio_path)
    if features is not None:
        features = features.reshape(1, 40, 174, 1)
        prediction = model.predict(features)
        print(f"Prediction: {'Real' if np.argmax(prediction) == 1 else 'Fake'}")

# Run Prediction & Evaluation
predict_audio(NEW_AUDIO_PATH)
evaluate_model()
